In [72]:
import pandas as pd 
from prophet import Prophet
import numpy as np 
import plotly.graph_objs as go

In [23]:
df = pd.read_csv("../data/root_ts.csv")
df = df.rename({"Unnamed: 0":"day_index"})
df.columns = ["day_index",'sales_quantity']

In [24]:
calender = pd.read_csv('../data/calendar.csv')
calender.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [25]:
df_merged = df.merge(calender[['d','date']], right_on='d', left_on='day_index')
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1913 entries, 0 to 1912
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   day_index       1913 non-null   object
 1   sales_quantity  1913 non-null   int64 
 2   d               1913 non-null   object
 3   date            1913 non-null   object
dtypes: int64(1), object(3)
memory usage: 74.7+ KB


In [27]:
df_merged.head()

,day_index,sales_quantity,d,date
0,d_1,32631,d_1,2011-01-29
1,d_2,31749,d_2,2011-01-30
2,d_3,23783,d_3,2011-01-31
3,d_4,25412,d_4,2011-02-01
4,d_5,19146,d_5,2011-02-02


In [37]:
df_merged['date'] = pd.to_datetime(df_merged['date'], format='%Y%m%d', errors='ignore')
df_merged = df_merged[['date','sales_quantity']]
df_merged.columns = ['ds','y']
df_merged.head()

,ds,y
0,2011-01-29,32631
1,2011-01-30,31749
2,2011-01-31,23783
3,2011-02-01,25412
4,2011-02-02,19146


In [60]:
df_merged.tail()

,ds,y
1908,2016-04-20,35343
1909,2016-04-21,35033
1910,2016-04-22,40517
1911,2016-04-23,48962
1912,2016-04-24,49795


In [101]:
model = Prophet(daily_seasonality=True)
model.fit(df_merged)

/Users/ericliu/Library/Python/3.8/lib/python/site-packages/prophet/forecaster.py:896: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



Initial log joint probability = -27.3885
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99        4390.2   0.000462918       88.5894      0.3819      0.3819      123   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     187       4393.31   7.18846e-05       123.548    4.57e-07       0.001      287  LS failed, Hessian reset 
     199       4393.44    0.00030333       97.6894           1           1      303   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     245       4393.83   0.000638502       324.915   8.284e-06       0.001      411  LS failed, Hessian reset 
     299       4394.17   7.02267e-05       51.0474           1           1      481   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     317        4394.3   0.000949357         209.5   1.318e-05       0.001      576  LS failed, Hessian rese

In [102]:
future = model.make_future_dataframe(periods=28)
future.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1941 entries, 0 to 1940
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   ds      1941 non-null   datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 15.3 KB


In [103]:
future.head()

,ds
0,2011-01-29
1,2011-01-30
2,2011-01-31
3,2011-02-01
4,2011-02-02


In [104]:
future.tail(28)

,ds
1913,2016-04-25
1914,2016-04-26
1915,2016-04-27
1916,2016-04-28
1917,2016-04-29
1918,2016-04-30
1919,2016-05-01
1920,2016-05-02
1921,2016-05-03
1922,2016-05-04


In [105]:
future.loc[1913].values[0]

numpy.datetime64('2016-04-25T00:00:00.000000000')

In [106]:
calender[calender['d'] == 'd_1941']

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
1940,2016-05-22,11617,Sunday,2,5,2016,d_1941,NaN,NaN,NaN,NaN,0,0,0


In [107]:
forecasting = model.predict(future)

/Users/ericliu/Library/Python/3.8/lib/python/site-packages/prophet/forecaster.py:896: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/ericliu/Library/Python/3.8/lib/python/site-packages/prophet/forecaster.py:896: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [108]:
# evaluation: 
sales_train_evaluaiton = pd.read_csv('../data/sales_train_evaluation.csv')
cols = sales_train_evaluaiton.columns[:]
sales_train_evluation_leaf = sales_train_evaluaiton.groupby('id').sum()
valid_root = sales_train_evluation_leaf[sales_train_evluation_leaf.columns[-28:]].sum(axis = 0)
valid_root = pd.DataFrame(valid_root,)
valid_root.columns = ['sales_quantity']
valid_root

,sales_quantity
d_1914,38793
d_1915,35487
d_1916,34445
d_1917,34732
d_1918,42896
d_1919,50429
d_1920,53032
d_1921,43181
d_1922,44314
d_1923,39601


In [109]:
calender[calender['d'] == 'd_1914']

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
1913,2016-04-25,11613,Monday,3,4,2016,d_1914,NaN,NaN,NaN,NaN,0,0,0


In [110]:
y_hat_valid_df = forecasting.tail(28)
y_hat_lower = y_hat_valid_df['yhat_lower']
y_hat_upper= y_hat_valid_df['yhat_upper']
y_hat =  y_hat_valid_df['yhat']
x = y_hat_valid_df['ds']
y = valid_root['sales_quantity']

In [111]:
def plot_forecast(x, y_hat, y_hat_lower, y_hat_upper, y):

    x = x.apply(lambda x: x.strftime('%Y-%m-%d'))

    fig = go.Figure([
        go.Scatter(
            name='y_hat',
            x=x,
            y=y_hat,
            mode='lines',
            line=dict(color='rgb(31, 119, 180)'),
        ),
        go.Scatter(
            name='Upper Bound',
            x=x,
            y=y_hat_upper,
            mode='lines',
            marker=dict(color="#444"),
            line=dict(width=0),
            showlegend=False
        ),
        go.Scatter(
            name='Lower Bound',
            x=x,
            y=y_hat_lower,
            marker=dict(color="#444"),
            line=dict(width=0),
            mode='lines',
            fillcolor='rgba(68, 68, 68, 0.3)',
            fill='tonexty',
            showlegend=False
        ),
        go.Scatter(
            name='y',
            x=x,
            y=y,
            mode='lines',
            line=dict(color='rgb(25, 190, 70)'),
        ),
    ])

    fig.update_layout(
        yaxis_title='Forecasted Sales Quantity',
        title='Comparison',
        hovermode="x"
    )

    
    fig.show()

In [112]:
plot_forecast(x, y_hat, y_hat_lower, y_hat_upper,y)